In [1]:
from wikitables import *
import mwclient
import mwparserfromhell as mwp
import logging

In [2]:
logger = logging.getLogger("wikitables")
logger.setLevel(logging.DEBUG)

In [3]:
# retrieve list of articles with tables in wikipedia
# get 10 articles each from a-z with appeneded random letter to act as random sample of tables
import string
import random

# articles = list()
# site = mwclient.Site('en.wikipedia.org')

# for letter in string.ascii_lowercase:
#     counter = 0
#     while (counter!=10):
#         for page in site.allpages(filterredir='nonredirects', prefix=letter+random.choice(string.ascii_lowercase)):
#             tables = mwp.parse(page.text()).filter_tags(matches=ftag('table'))
#             if tables:
#                 t = import_tables(page.name)
#                 if t:
#                     print(page.name)
#                     articles.append(page.name)
#                     counter+=1
#                     break

In [4]:
# # save the articles with tables
# import pickle

# with open("articles.txt", "wb") as fp:
#     pickle.dump(articles, fp)

In [5]:
# import the saved articles
import pickle
with open("articles.txt", "rb") as fp:
    articles = list(set(pickle.load(fp)))

In [6]:
# extract tables and features
t = import_tables("Ia Ora 'O Tahiti Nui")

Ia Ora 'O Tahiti Nui[0]


In [7]:
t[0].rows

[{'Official Text in Tahitian': ʻUa rahu te atua i tōʻu ʻāiʻa
  Hono noʻanoʻa o te motu rau
  Heihei i te pua riʻi au ē
  E firi nape mōrohi ʻore
  ʻO tāʻu īa e faʻateniteni nei
  Tē tūoro nei te reo here
  O te huia
  ʻA hiʻi i tō aroha
  ʻIa ora ʻo Tahiti Nui ē, 'Translation': God created my homeland
  Fragrant garland laid across the many islands
  Ode to the little flowers we love
  It's a never-fading wreath of sennit
  That I celebrate today
  Listen to your children's voices
  Calling us to cherish the beloved
  Long live Tahiti Nui !}]

In [8]:
# extract RDF triples